In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install h5netcdf # to install NWP data
import h5netcdf # to install NWP data
import xarray as xr # to install NWP data
import s3fs # to install NWP data

import os
import plotly.express as px # for price region plotting on map
from collections import defaultdict

import warningsc
warnings.simplefilter('ignore') # Ignoring thrid parting warrnings

In [ ]:
# fetching hourly numerical weather prediction
date1 = '2000-01-01'
date2 = '2000-12-31'
mydates = pd.date_range(date1, date2).astype('str').tolist()
missing_dates = ['2000-05-14', '2000-09-26']
for c in missing_dates:
    if c in mydates:
        mydates.remove(c)
def remove_punc(text):
    no_punc= ''.join([c for c in text if c != '-'])
    return no_punc
dates = list(map(remove_punc, mydates))
months = [c[4:6] for c in dates]
combined = list(zip(dates, months))
def NWP_data(value):
    A= None
    fs_s3 = s3fs.S3FileSystem(anon=True)
    s3path = 'greenlytics-public/forecasting-competition/releases/Task0/'+value[1]+'/'+value[0]+'T00Z.nc'
    remote_file_obj = fs_s3.open(s3path, mode='rb')
    A= eval("xr.open_dataset(remote_file_obj, engine= 'h5netcdf')")
    #exec("return A{}".format(combined[0][0][4:]))
    return A
for i, value in enumerate(combined):
    try:
        exec('ds{}= NWP_data(value)'.format(i))
        print('ds{} done'.format(i))
    except:
        print('Could not found value')

In [ ]:
os.listdir()

In [ ]:
power_data= pd.read_csv('windpower_task0_updated.csv') #fetching power data
turbine_data= pd.read_csv('windturbines.csv') #fetching turbines credentials

In [ ]:
power_data.head()

In [ ]:
turbine_data.head()

So there are 4004 turbines with columns  'Unnamed: 0',   'Wind turbine ID', 'Terrain height [m]',
        'Nacelle height [m]', 'Rotor diameter [m]',   'Max power [MW]',
       'Price region', 'Installation date', 'Longitude', 'Latitude'.

In [ ]:
# Satellite view of windturbines
fig = px.scatter_mapbox(turbine_data, lat="Latitude", lon="Longitude", hover_name="Wind turbine ID", hover_data=['Installation date','Max power [MW]']
                        , zoom=3, height=300, color= 'Price region')
fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
power_data.rename(columns={'Unnamed: 0' : 'Timestamp'}, inplace= True) # changing name of times column 

In [ ]:
power_data.head()

In [ ]:
values= []
for region in ['SE1', 'SE2', 'SE3', 'SE4']:
    for charac in ['Terrain height [m]', 'Nacelle height [m]', 'Rotor diameter [m]', 'Max power [MW]']:
        value= turbine_data[turbine_data['Price region'] == region ][charac].mean()
        values.append((region , value))

values_dict= defaultdict(list)
for region, value in values:
    values_dict[region].append(value)
mean_df = pd.DataFrame(data= values_dict, index= ['Mean_Terrain height [m]', 'Mean_Nacelle height [m]', 'Mean_Rotor diameter [m]', 'Mean_Max power [MW]'])
mean_df

In [ ]:
power_data.info()

In [ ]:
power_data.set_index('Timestamp', inplace= True)
power_data.describe()

In [ ]:
turbine_data.info()

We have 24 values for each coordinate. We have 169 * 71 different locations. We have 10 different forecasted value for each location.

In [ ]:
power_data.reset_index(inplace= True)
power_data.head()

In [ ]:
power_data.Timestamp = pd.to_datetime(power_data.Timestamp)
power_data['Month'] = power_data.Timestamp.dt.month
power_data['Hour'] = power_data.Timestamp.dt.hour
power_data['Day']= power_data.Timestamp.dt.day
power_data.head()

In [ ]:
print(len(power_data.Day.value_counts()), len(power_data.Month.value_counts()))

In [ ]:
power_data.Timestamp

In [ ]:
turbine_data[['Latitude', 'Longitude']]

In [ ]:
a = ds0.to_dataframe()
a = a.reset_index()
a.head()

In [ ]:
a = a[a.ensemble_member == 0]
a.head()

In [ ]:
a = a[a.time == ' 2000-01-01 00:00:00']

In [ ]:
NWP_Coordinates = a[['latitude','longitude']]